In [3]:
import requests
from bs4 import BeautifulSoup
import urllib3
import re
import pandas as pd

def top_search_result_wiki_company(company_name):
  subscription_key = "c5a282b2cba14ce5a3ad451c3575bd91"
  search_url = "https://api.bing.microsoft.com/v7.0/search"
  search_term = company_name + " company stock wikipedia"
  headers = {"Ocp-Apim-Subscription-Key": subscription_key}
  params = {"q": search_term, "textDecorations": True, "textFormat": "HTML", "mkt":"en-US"}
  response = requests.get(search_url, headers=headers, params=params)
  response.raise_for_status()
  search_results = response.json()
  rows = "\n".join(["""<tr>
                       <td><a href=\"{0}\">{1}</a></td>
                       <td>{2}</td>
                     </tr>""".format(v["url"], v["name"], v["snippet"])
                  for v in search_results["webPages"]["value"]])

  soup = BeautifulSoup("<table>{0}</table>".format(rows))
  links = []
  for link in soup.findAll('a'):
      links.append(link.get('href'))
  
  df = pd.read_html("<table>{0}</table>".format(rows))[0].rename(columns={0: "site", 1: "summary"})
  df["url"] = links
  df["rank"] = range(1, len(df)+1)
  df["company"] = company_name
  return df




In [2]:
top_search_result_wiki_company("Engie SA")

,site,summary,url,rank,company
0,Engie - Wikipedia,Engie SA is a French multinational utility com...,https://en.wikipedia.org/wiki/Engie,1,Engie SA
1,World Energy Actor | ENGIE,ENGIE supports industries in increasing their ...,https://www.engie.com/en,2,Engie SA
2,Electrabel - Wikipedia,Electrabel SA is a Belgian energy corporation....,https://en.wikipedia.org/wiki/Electrabel,3,Engie SA
3,E-CL - Wikipedia,Engie. Website. www.e-cl.cl. E-CL is a Chilean...,https://en.wikipedia.org/wiki/E-CL,4,Engie SA
4,Eni - Wikipedia,Eni was founded in 1953 from an existing compa...,https://en.wikipedia.org/wiki/Eni,5,Engie SA
5,Who we are | ENGIE,Jean-Pierre Clamadieu opens the Conference of ...,https://www.engie.com/en/group/who-we-are,6,Engie SA
6,Governance | ENGIE,As the energy transition and digital transform...,https://www.engie.com/en/group/who-are-we/gove...,7,Engie SA
7,The History of ENGIE | Group | ENGIE,The history of ENGIE begins in the first half ...,https://www.engie.com/en/group/history-engie-g...,8,Engie SA
8,Suez (company) - Wikipedia,Suez S.A. was a leading French multinational c...,https://en.wikipedia.org/wiki/Suez_(company),9,Engie SA
9,ENGIE :: France :: OpenCorporates,Free and open company data on France company E...,https://opencorporates.com/companies/fr/542107651,10,Engie SA


In [ ]:
from tqdm import tqdm
df_list = []

for company in tqdm(companies_list):
  df = top_search_result_wiki_company(company)
  df_list.append(df)
df_merged = pd.concat(df_list)
df_merged.to_csv("companies_wiki.csv")
